In [1]:
import os
import sys
import time
import yaml
import pandas as pd
import numpy as np

with open('../../config.local.yaml', 'r') as f:
    local_config = yaml.safe_load(f)

LOCAL_PATH = local_config['LOCAL_PATH']

sys.path.append(os.path.join(LOCAL_PATH, "src/python"))

from llm import get_response

LLM_OVERWRITE = False

/Users/ekung/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ekung/Library/Python/3.9/lib/python/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
meetings_df = pd.read_csv(os.path.join(LOCAL_PATH, "intermediate_data/cpc/meetings-manifest.csv"))
DATES = sorted(list(meetings_df['date']))

In [3]:
PROMPT = """
--- AGENDA ITEM ----

{}

--- MINUTES OF DISCUSSION ----

{}

--- PROMPT ----

I just gave you two documents related to a Los Angeles City Planning Commission (CPC) hearing.

The first document is the agenda item to be discussed, with requested actions.

The second document is the minutes of the discussion, the proposed motion by the CPC, the votes on the motion by the CPC members, and whether the motion ultimately passed.

Please return a response in the following format:

---- YOUR RESPONSE FORMAT ----
RELATED CASES:
<A comma separated list of relevant planning department case numbers>

SUMMARY OF AGENDA ITEM:
<A summary of the agenda item to be discussed>

SUMMARY OF CPC DELIBERATIONS:
<A summary of the deliberations of the CPC.>

SUMMARY OF CPC MOTION:
<A summary of the motion voted on by the CPC>

MOVED:
<Which commission member moved the motion? If multiple motions were made, use only the last motion. If no motion was made, say "N/A".>

SECONDED:
<Which commission member seconded the motion? If multiple motions were made, use only the last motion. If no motion was made, say "N/A".>

AYES:
<A comma separated list of commission members who voted for the motion. If multiple motions were made, use only the last motion. If no one voted for, say "NONE". If no motion was made, say "N/A".>

NAYS:
<A comma separated list of commission members who voted against the motion. If multiple motions were made, use only the last motion. If no one voted against, say "NONE". If no motion was made, say "N/A".>

ABSTAINED:
<A comma separated list of commission members who abstained from voting on the motion. If multiple motions were made, use only the last motion. If no one abstained, say "NONE". If no motion was made, say "N/A".>

RECUSED:
<A comma separated list of commission members who were recused from voting on the motion. If multiple motions were made, use only the last motion. If no one was recused, say "NONE". If no motion was made, say "N/A".>

ABSENT:
<A comma separated list of commission members who were absent. If multiple motions were made, use only the last motion. If no one was absent, say "NONE". If no motion was made, say "N/A".>

VOTE RESULT:
<Did the motion pass or fail? Your only options are MOTION PASSED, MOTION FAILED, N/A>

RESULT OF APPEAL:
<If the agenda item involved an appeal, say whether the appeal was granted or denied. Your only options are: APPEAL GRANTED, APPEAL GRANTED IN PART, APPEAL DENIED, NO APPEAL, DELIBERATIONS CONTINUED TO FUTURE DATE, APPLICATION WITHDRAWN>

IMPLICATION FOR PROJECT:
<What is the implication of the vote for the original requested actions? Your only options are: APPROVED, APPROVED IN PART OR WITH MODIFICATIONS, DENIED, DELIBERATIONS CONTINUED TO FUTURE DATE, APPLICATION WITHDRAWN>
""" 

In [15]:
out = PROMPT.replace('{}', "<<agenda item text>>", 1)
out = out.replace('{}', "<<minutes text for item>>", 1)
out = out.strip()
out = out.replace('\n', '\\\\ \n')
lines = out.split('\n')

with open(os.path.join(LOCAL_PATH, 'figures', 'minutes_prompt_1.tex'), 'w') as f:
    out1 = '\n'.join(lines[:len(lines)//2+7]).strip()
    out1 = out1[:-6]
    f.write(out1)
with open(os.path.join(LOCAL_PATH, 'figures', 'minutes_prompt_2.tex'), 'w') as f:
    out2 = '\n'.join(lines[len(lines)//2+7:]).strip()
    f.write(out2)

In [ ]:
t0 = time.time()
for date in DATES:
    year = date[0:4]
    print(date)
    PATH = os.path.join(LOCAL_PATH, f"intermediate_data/cpc/{year}/{date}")
    agenda_df = pd.read_pickle(os.path.join(PATH, 'agenda-items.pkl'))
    minutes_df = pd.read_pickle(os.path.join(PATH, 'minutes-items.pkl'))
    
    df = []
    for j, row2 in agenda_df.iterrows():
        item_no = row2['item_no']
        title = row2['title']
        is_casenum = row2['is_casenum']
        if is_casenum:
            print(f"{item_no}... ", end='')
            agenda_content = row2['content']
            minutes_content = minutes_df.loc[minutes_df['item_no']==item_no].iloc[0]['content']
            minutes_start_line = minutes_df.loc[minutes_df['item_no']==item_no].iloc[0]['start_line']
            minutes_end_line = minutes_df.loc[minutes_df['item_no']==item_no].iloc[0]['end_line']
            prompt = PROMPT.format(agenda_content, minutes_content)
            response = get_response(prompt, overwrite=LLM_OVERWRITE)
            msg = response['message']
            perplexity = response['perplexity']
            df.append({
                'year': year,
                'date': date,
                'item_no': item_no,
                'title': title,
                'agenda_content': agenda_content,
                'minutes_content': minutes_content,
                'minutes_start_line': minutes_start_line,
                'minutes_end_line': minutes_end_line,
                'prompt': prompt,
                'response': msg,
                'perplexity': perplexity
            })
    print('')
    df = pd.DataFrame.from_dict(df)
    if len(df)>0:
        df.to_pickle(os.path.join(PATH, 'minutes-summaries.pkl'))
            
t1 = time.time()
print(f"Elapsed time: {(t1-t0)/60} minutes.")
